In [1]:
import pytimber
ldb = pytimber.LoggingDB()

Specify the output file and the times you want to download. Timber and pyTimber conflict with regards to the the times, probably because of winter and summertime. If you want to have data stored in Timber from 00:00 to 01:00, you might to request either from 01:00 to 02:00 or even 02:00 to 03:00. We have to account for this shift later on.

In [100]:
filename = '../Data_Raw/Nov2018.csv'

In [101]:
import pandas as pd

t1 = '2018-11-01 02:00:00.000'
t2 = '2018-12-01 02:00:00.000'
timeshift_to_timber = pd.Timedelta(hours=2)
replace_file = True

Now select all parameters you are interested in.

In [102]:
time = 'Timestamp (UTC_TIME)'
parameters = ['IP.NSRCGEN:BIASDISCAQNV', 'IP.NSRCGEN:GASSASAQN', 'IP.NSRCGEN:SOURCEHTAQNI', 'IP.SOLCEN.ACQUISITION:CURRENT', 'IP.SOLEXT.ACQUISITION:CURRENT', 'IP.NSRCGEN:OVEN1AQNP', 'IP.NSRCGEN:OVEN2AQNP', 'ITF.BCT25:CURRENT']
parameters += ['IP.NSRCGEN:GASAQN', 'IP.SAIREM2:FORWARDPOWER', 'IP.SOLINJ.ACQUISITION:CURRENT']
columns_to_load = parameters

In [103]:
result = ldb.get(columns_to_load, t1, t2, unixtime=False)

In [104]:
import pandas as pd
from os import path

def load_existing_data(filename):
    if not path.exists(filename):
        print("The file {} does not yet exist, we will create a new one".format(filename))
        return pd.DataFrame(columns=[time])
    
    print("Loading data from {}. We will only append new columns".format(filename))
    df = pd.read_csv(filename)
    return df

if replace_file:
    df = pd.DataFrame(columns=[time])
else:
    df = load_existing_data(filename)
    
df.set_index(time, inplace = True)

In [105]:
def check_duplicate_times(time_series):
    x = time_series.duplicated()
    count = x[x].count()
    if count > 0:
        print("Time duplicates exist!")

for parameter, values in result.items():
    print("For column {} {} datapoints exist.".format(parameter, len(values[1])))
    
    if parameter in df.columns:
        print("Parameter {} is already in the data frame. There it has {} values. In the newly retrieved dataset it has {} values. Skipping.".format(parameter, df[parameter].count(), len(values[1])))
        continue
    
    df_column = pd.DataFrame(columns=[time, parameter])
    df_column[time] = values[0]
    check_duplicate_times(df_column[time])
    df_column[parameter] = values[1]
    
    df_column.set_index(time, inplace = True)
    if len(df_column.index) > 0:
        df_column.index -= timeshift_to_timber

    df = df.join(df_column, how='outer')
df = df.reindex(sorted(df.columns), axis=1)

For column IP.NSRCGEN:BIASDISCAQNV 9805 datapoints exist.
For column IP.NSRCGEN:GASAQN 11497 datapoints exist.
For column IP.NSRCGEN:GASSASAQN 8650 datapoints exist.
For column IP.NSRCGEN:OVEN1AQNP 8498 datapoints exist.
For column IP.NSRCGEN:OVEN2AQNP 8621 datapoints exist.
For column IP.NSRCGEN:SOURCEHTAQNI 60593 datapoints exist.
For column IP.SAIREM2:FORWARDPOWER 2142763 datapoints exist.
For column IP.SOLCEN.ACQUISITION:CURRENT 2001 datapoints exist.
For column IP.SOLEXT.ACQUISITION:CURRENT 22668 datapoints exist.
For column IP.SOLINJ.ACQUISITION:CURRENT 22012 datapoints exist.
For column ITF.BCT25:CURRENT 367687 datapoints exist.


If the timeindex is duplicated, we will only keep the first occurence.

In [106]:
df = df[~df.index.duplicated(keep='first')].copy()

To be consistent with Timber, we convert the times to UTC.

In [107]:
df

,IP.NSRCGEN:BIASDISCAQNV,IP.NSRCGEN:GASAQN,IP.NSRCGEN:GASSASAQN,IP.NSRCGEN:OVEN1AQNP,IP.NSRCGEN:OVEN2AQNP,IP.NSRCGEN:SOURCEHTAQNI,IP.SAIREM2:FORWARDPOWER,IP.SOLCEN.ACQUISITION:CURRENT,IP.SOLEXT.ACQUISITION:CURRENT,IP.SOLINJ.ACQUISITION:CURRENT,ITF.BCT25:CURRENT
Timestamp (UTC_TIME),,,,,,,,,,,
2018-11-01 00:00:53.255,NaN,10.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-11-01 00:01:40.055,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-11-01 00:02:54.455,NaN,NaN,NaN,NaN,0.004,NaN,NaN,NaN,NaN,NaN,NaN
2018-11-01 00:03:17.255,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.42,NaN,-0.0008
2018-11-01 00:03:40.055,NaN,NaN,NaN,NaN,NaN,-0.0254,NaN,NaN,NaN,NaN,NaN
2018-11-01 00:04:54.455,NaN,NaN,NaN,0.002,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-11-01 00:05:54.455,NaN,10.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-11-01 00:07:37.655,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-11-01 00:07:55.655,NaN,NaN,NaN,NaN,0.003,NaN,NaN,NaN,NaN,NaN,NaN


And save the output to the file.

In [108]:
df.to_csv(filename)